# 🌊🌀 WakingOrca Quantum v2.0 - ONE-CLICK ARC PRIZE 2025 SOLVER

**🎯 GOAL**: Maximum accuracy in 20-minute runtime

**🔬 QUANTUM ADVANTAGE**: NSM Attractor Basin Detection + Higher-Dimensional Projection

**⚡ USAGE**: Just click "Run All" in Kaggle - submission.json generated automatically

**👤 Authors**: Ryan Cardwell & Claude | **📅 Date**: November 2025

In [ ]:
# Cell 1: Imports & Configuration (ONE-CLICK OPTIMIZED)
import json
import numpy as np
import time
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Set, Any
from dataclasses import dataclass, field
from collections import defaultdict, Counter
from itertools import product, combinations
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import svd
import hashlib
import warnings
warnings.filterwarnings('ignore')

@dataclass
class Config:
    """ONE-CLICK Configuration for 20-minute max accuracy run"""
    # Paths
    data_path: str = '/kaggle/input/arc-prize-2025'
    output_path: str = '/kaggle/working'
    
    # Time management (20 MINUTES TOTAL)
    total_time_budget: float = 1200.0  # 20 minutes
    time_per_task: float = 4.0  # 4 seconds per task (240 tasks × 4s = 960s + 240s overhead)
    
    # Quantum advantage parameters
    use_nsm_basins: bool = True  # Use attractor basin detection
    use_higher_dims: bool = True  # Use 36D projection
    dimension: int = 36  # Feature space dimension
    
    # Solver parameters (OPTIMIZED FOR SPEED)
    max_solutions_per_task: int = 2
    recursion_depth: int = 4  # Reduced from 6 for speed
    superposition_branches: int = 10  # Reduced from 15 for speed
    
    # Quantum parameters
    collapse_threshold: float = 0.4  # Lower threshold = more solutions
    entanglement_detection: bool = False  # Disabled for speed

config = Config()
print("="*60)
print("🌊🌀 WAKINGORCA QUANTUM v2.0 - ONE-CLICK SOLVER")
print("="*60)
print(f"⏱️  Time budget: {config.total_time_budget}s ({config.total_time_budget/60:.1f} min)")
print(f"🎯 Time per task: {config.time_per_task}s")
print(f"🌀 Quantum Advantage: NSM={'ON' if config.use_nsm_basins else 'OFF'}, Dims={config.dimension}D")
print(f"🔧 Recursion: {config.recursion_depth}, Branches: {config.superposition_branches}")
print("="*60)

In [ ]:
# Cell 2: Data Loading
def load_arc_data(config: Config) -> Tuple[Dict, Dict]:
    training_tasks = {}
    test_tasks = {}
    
    data_path = Path(config.data_path)
    
    # Load training
    train_path = data_path / 'arc-agi_training_challenges.json'
    if train_path.exists():
        with open(train_path, 'r') as f:
            training_tasks = json.load(f)
    
    # Load test  
    test_path = data_path / 'arc-agi_test_challenges.json'
    if test_path.exists():
        with open(test_path, 'r') as f:
            test_tasks = json.load(f)
    
    print(f"📚 Loaded {len(training_tasks)} training tasks, {len(test_tasks)} test tasks")
    return training_tasks, test_tasks

training_tasks, test_tasks = load_arc_data(config)

In [ ]:
# Cell 3: NSM Dimensional Projector (QUANTUM ADVANTAGE CORE)
class DimensionalProjector:
    """Project 2D grids into 36D space to reveal hidden patterns"""
    
    def __init__(self, dimension: int = 36):
        self.dimension = dimension
    
    def project(self, grid: np.ndarray) -> np.ndarray:
        """Fast projection into higher dimensions"""
        features = []
        
        # D1-3: Spatial
        features.extend([grid.shape[0], grid.shape[1], grid.size])
        
        # D4-6: Color stats
        features.extend([len(np.unique(grid)), np.mean(grid), np.std(grid)])
        
        # D7-12: Symmetry (6 transforms)
        for transform in [grid, np.rot90(grid), np.rot90(grid,2), np.fliplr(grid), np.flipud(grid)]:
            if transform.shape == grid.shape:
                features.append(np.sum(transform == grid) / grid.size)
            else:
                features.append(0.0)
        features.append(0.0)  # Pad to 6
        
        # D13-18: Topological
        features.extend([
            len(np.unique(grid)),  # Components proxy
            np.sum(np.diff(grid, axis=0) != 0) if grid.shape[0] > 1 else 0,  # H boundaries
            np.sum(np.diff(grid, axis=1) != 0) if grid.shape[1] > 1 else 0,  # V boundaries
            0, 0, 0  # Placeholder
        ])
        
        # D19-24: Information
        flat = grid.flatten()
        counts = np.bincount(flat)
        probs = counts[counts > 0] / len(flat)
        entropy = -np.sum(probs * np.log2(probs + 1e-10))
        features.extend([entropy, np.max(probs), np.min(probs), np.median(probs), 0, 0])
        
        # D25-30: Structure
        features.extend([
            np.sum(np.diff(flat) != 0) / len(flat),  # Change rate
            grid[0,0], grid[-1,-1],  # Corners
            grid[grid.shape[0]//2, grid.shape[1]//2] if grid.size > 0 else 0,  # Center
            0, 0
        ])
        
        # D31-36: Nonlinear
        features.extend([f*f for f in features[:6]])
        
        # Ensure exactly dimension features
        feature_vec = np.array(features[:self.dimension])
        if len(feature_vec) < self.dimension:
            feature_vec = np.pad(feature_vec, (0, self.dimension - len(feature_vec)))
        
        return feature_vec

print("🌀 Dimensional projector initialized")

In [ ]:
# Cell 4: NSM Attractor Basin Detector (QUANTUM ADVANTAGE)
@dataclass
class AttractorBasin:
    name: str
    center: np.ndarray
    radius: float
    rule: str
    examples: List[Tuple[np.ndarray, np.ndarray]] = field(default_factory=list)

class AttractorBasinDetector:
    """Detect great attractor basins in puzzle space"""
    
    def __init__(self, dimension: int = 36):
        self.dimension = dimension
        self.projector = DimensionalProjector(dimension)
        self.basins: List[AttractorBasin] = []
    
    def detect_basins(self, examples: List[Dict]) -> List[AttractorBasin]:
        """Fast basin detection"""
        if len(examples) < 1:
            return []
        
        # Project examples
        transforms = []
        for ex in examples:
            inp = np.array(ex['input'])
            out = np.array(ex['output'])
            inp_proj = self.projector.project(inp)
            out_proj = self.projector.project(out)
            transforms.append((out_proj - inp_proj, inp, out))
        
        # Simple clustering: single basin if all similar
        if len(transforms) == 1:
            t, inp, out = transforms[0]
            rule = self._infer_rule(inp, out)
            self.basins = [AttractorBasin(
                name="Basin_0",
                center=t,
                radius=0.0,
                rule=rule,
                examples=[(inp, out)]
            )]
        else:
            # Average all transforms as single basin (fast)
            transform_vecs = [t for t, _, _ in transforms]
            center = np.mean(transform_vecs, axis=0)
            radius = np.max([np.linalg.norm(t - center) for t in transform_vecs])
            rule = self._infer_rule(transforms[0][1], transforms[0][2])
            self.basins = [AttractorBasin(
                name="Basin_0",
                center=center,
                radius=radius,
                rule=rule,
                examples=[(inp, out) for _, inp, out in transforms]
            )]
        
        return self.basins
    
    def _infer_rule(self, inp: np.ndarray, out: np.ndarray) -> str:
        """Fast rule inference"""
        if inp.shape != out.shape:
            return "scale"
        if np.array_equal(out, np.rot90(inp)):
            return "rotate_90"
        if np.array_equal(out, np.fliplr(inp)):
            return "flip_h"
        if np.array_equal(out, np.flipud(inp)):
            return "flip_v"
        return "complex"
    
    def predict_transform(self, test_input: np.ndarray) -> Tuple[str, List[np.ndarray]]:
        """Predict transformation"""
        if not self.basins:
            return "unknown", [test_input]
        
        basin = self.basins[0]
        rule = basin.rule
        
        # Apply rule
        if rule == "rotate_90":
            return rule, [np.rot90(test_input)]
        elif rule == "flip_h":
            return rule, [np.fliplr(test_input)]
        elif rule == "flip_v":
            return rule, [np.flipud(test_input)]
        else:
            # Try to apply transformation from example
            if basin.examples:
                ex_in, ex_out = basin.examples[0]
                # Try color mapping if same shape
                if test_input.shape == ex_in.shape == ex_out.shape:
                    return "color_map", [self._apply_color_map(test_input, ex_in, ex_out)]
            return "unknown", [test_input]
    
    def _apply_color_map(self, test, ex_in, ex_out):
        """Apply learned color mapping"""
        color_map = {}
        for i in range(ex_in.shape[0]):
            for j in range(ex_in.shape[1]):
                color_map[int(ex_in[i,j])] = int(ex_out[i,j])
        
        result = test.copy()
        for i in range(result.shape[0]):
            for j in range(result.shape[1]):
                val = int(result[i,j])
                if val in color_map:
                    result[i,j] = color_map[val]
        return result

print("🌀 Attractor basin detector initialized")

In [ ]:
# Cell 5: Causal Primitives
class CausalPrimitives:
    @staticmethod
    def identity(g): return g.copy()
    @staticmethod
    def rotate_90(g): return np.rot90(g)
    @staticmethod
    def rotate_180(g): return np.rot90(g, 2)
    @staticmethod
    def rotate_270(g): return np.rot90(g, 3)
    @staticmethod
    def flip_h(g): return np.fliplr(g)
    @staticmethod
    def flip_v(g): return np.flipud(g)
    @staticmethod
    def transpose(g): return g.T
    @staticmethod
    def invert(g): return (g.max() - g) if g.max() > 0 else g

primitives = CausalPrimitives()
print("🔧 Primitives loaded")

In [ ]:
# Cell 6: Quantum Superposition (SIMPLIFIED FOR SPEED)
class QuantumSuperposition:
    def __init__(self, config):
        self.config = config
        self.states = []  # (solution, amplitude) pairs
    
    def add_state(self, solution: np.ndarray, amplitude: float):
        self.states.append((solution, amplitude))
        # Keep only top N
        if len(self.states) > self.config.superposition_branches:
            self.states.sort(key=lambda x: x[1], reverse=True)
            self.states = self.states[:self.config.superposition_branches]
    
    def collapse(self) -> List[np.ndarray]:
        if not self.states:
            return []
        self.states.sort(key=lambda x: x[1], reverse=True)
        return [s[0] for s in self.states[:self.config.max_solutions_per_task]]

print("🌀 Superposition manager loaded")

In [ ]:
# Cell 7: Fast Recursive Solver
class FastRecursiveSolver:
    def __init__(self, config):
        self.config = config
        self.primitives = CausalPrimitives()
    
    def solve(self, input_grid: np.ndarray, examples: List[Dict], depth: int = 0) -> QuantumSuperposition:
        superposition = QuantumSuperposition(self.config)
        
        if depth >= self.config.recursion_depth:
            return superposition
        
        # Try primitives
        for name, method in [
            ('rotate_90', self.primitives.rotate_90),
            ('rotate_180', self.primitives.rotate_180),
            ('flip_h', self.primitives.flip_h),
            ('flip_v', self.primitives.flip_v),
            ('transpose', self.primitives.transpose),
        ]:
            try:
                result = method(input_grid)
                amplitude = self._calc_amplitude(result, examples)
                superposition.add_state(result, amplitude)
                
                # Recurse if promising
                if amplitude > 0.5 and depth < 2:  # Limited recursion for speed
                    sub = self.solve(result, examples, depth + 1)
                    for sol in sub.collapse():
                        superposition.add_state(sol, amplitude * 0.8)
            except:
                pass
        
        return superposition
    
    def _calc_amplitude(self, result: np.ndarray, examples: List[Dict]) -> float:
        if not examples:
            return 0.5
        
        best_sim = 0.0
        for ex in examples:
            if 'output' in ex:
                out = np.array(ex['output'])
                if out.shape == result.shape:
                    sim = np.sum(out == result) / result.size
                    best_sim = max(best_sim, sim)
        
        return best_sim if best_sim > 0 else 0.5

print("⚡ Fast solver loaded")

In [ ]:
# Cell 8: QUANTUM ADVANTAGE ORCHESTRATOR
class QuantumOrchestr